In [2]:
import numpy as np
import pandas as pd

5 nodes 

In [3]:
# Function to generate a dataset and its ground truth causal matrix
def generate_dataset(n, nan_ratio=0.1, phi1=0.8, phi2=0.7):

    # Initialize latent confounders
    latent1 = np.zeros(n)
    latent2 = np.zeros(n)
    latent1[0] = np.random.normal()
    latent2[0] = np.random.normal()

    # Generate latent confounders using AR(1) processes
    for t in range(1, n):
        latent1[t] = phi1 * latent1[t-1] + np.random.normal()
        latent2[t] = phi2 * latent2[t-1] + np.random.normal()

    # Initialize system variables y1 to y5
    y1 = np.zeros(n)
    y2 = np.zeros(n)
    y3 = np.zeros(n)
    y4 = np.zeros(n)
    y5 = np.zeros(n)

    # Generate y1 influenced by y2 (one time lag) and latent1
    y1[0] = np.random.normal()
    for t in range(1, n):
        y1[t] = 0.5 * y2[t-1] + 0.6 * latent1[t] + np.random.normal()

    # Generate y2 using AR(1) process (y2 causes itself over time)
    y2[0] = np.random.normal()
    for t in range(1, n):
        y2[t] = 0.7 * y2[t-1] + 0.8 * latent2[t] + np.random.normal()

    # Generate y3 influenced by latent2
    y3[0] = np.random.normal()
    for t in range(1, n):
        y3[t] = 0.8 * latent2[t] + np.random.normal()

    # Generate y4 influenced by y3 (one time lag) and latent1
    y4[0] = np.random.normal()
    for t in range(1, n):
        y4[t] = 0.8 * y3[t-1] + 0.5 * latent1[t] + np.random.normal()

    # Generate y5 influenced by y4 (contemporaneous), self-lag, and latent2
    y5[0] = np.random.normal()
    for t in range(1, n):
        y5[t] = 0.6 * y5[t-1] + 0.9 * y4[t] + np.random.normal()

    # Introduce NaN values randomly
    nan_indices = np.random.choice(n, size=int(n * nan_ratio), replace=False)
    for y in [y1, y2, y3, y4, y5]:
        y[nan_indices] = np.nan

    # Combine into DataFrame
    data = pd.DataFrame({'y1': y1, 'y2': y2, 'y3': y3, 'y4': y4, 'y5': y5})

    # Ground truth causal matrix
    ground_truth = np.array([
        # y1 caused by y2[t-1] and latent1
        [['', ''],  # y1 not influencing anything
         ['', ''],  # y1 caused by y2[t-1]
         ['', ''],     # No relation with y3
         ['o-o', ''],  # y1 caused by latent1
         ['', '']],    # No relation with y5

        # y2 causes itself over time (AR(1)) and causes y1[t-1]
        [['', '-->'],    # y2 not caused by y1
         ['', '-->'],  # y2 causes itself at lag 1
         ['o-o', ''],  # y2 caused by latent2
         ['', ''],    # No relation with y4
         ['', '']],   # No relation with y5

        # y3 caused by latent2, causes y4[t-1]
        [['', ''],    # No relation with y1
         ['o-o', ''], # y3 caused by latent2
         ['', ''],    # y3 not influenced by itself
         ['', '-->'], # y3[t-1] causes y4
         ['', '']],   # No relation with y5

        # y4 caused by y3[t-1] and latent1, causes y5 contemporaneously
        [['o-o', ''],    # No relation with y1
         ['', ''],    # No relation with y2
         ['', ''], # y4 caused by y3[t-1]
         ['-->', ''],    # y4 not influenced by itself
         ['-->', '']], # y4 causes y5 contemporaneously

        # y5 caused by y4 contemporaneously and self at lag
        [['', ''],    # No relation with y1
         ['', ''],    # No relation with y2
         ['o-o', ''], # y5 caused by latent2
         ['<--', ''], # y5 caused by y4 contemporaneously
         ['', '-->']]  # y5 causes itself at lag
    ])

    return data, ground_truth

# Example usage:
n = 300  # Length of the time series
nan_ratio = 0.1  # Ratio of NaN values

# Generate dataset and ground truth
data, ground_truth = generate_dataset(n, nan_ratio)

# Show the generated dataset and ground truth
print(data.head())


         y1        y2        y3        y4        y5
0  0.973723 -0.277603  1.523919  0.528692  0.116470
1  0.734256 -1.618485  1.927383  1.655883  0.853793
2  0.590121 -1.672467 -0.657566  3.388754  4.274545
3 -1.728108 -0.634837 -0.573889 -0.500856  1.259387
4 -0.708397  0.229302 -1.030391  1.078399  3.217724


In [ ]:


# Function to generate the dataset and its ground truth causal matrix
def generate_causal_dataset(n, nan_ratio=0.1, phi_latent=0.8):
    # Initialize latent confounder
    latent1 = np.zeros(n)
    latent1[0] = np.random.normal()

    # Generate latent confounder using an AR(1) process
    for t in range(1, n):
        latent1[t] = phi_latent * latent1[t-1] + np.random.normal()

    # Initialize variables y1 to y5
    y1 = np.zeros(n)
    y2 = np.zeros(n)
    y3 = np.zeros(n)
    y4 = np.zeros(n)
    y5 = np.zeros(n)

    # Generate y2 influenced by latent1
    y2[0] = np.random.normal()
    for t in range(1, n):
        y2[t] = 0.6 * latent1[t] + np.random.normal()

    # Generate y1 influenced by y2[t] (contemporaneous)
    y1[0] = np.random.normal()
    for t in range(1, n):
        y1[t] = 0.5 * y2[t] + np.random.normal()

    # Generate y3 influenced by y1[t-1]
    y3[0] = np.random.normal()
    for t in range(1, n):
        y3[t] = 0.7 * y1[t-1] + np.random.normal()

    # Generate y4 influenced by y2[t-1] and y3[t] (contemporaneous)
    y4[0] = np.random.normal()
    for t in range(1, n):
        y4[t] = 0.5 * y2[t-1] + 0.8 * y3[t] + np.random.normal()

    # Generate y5 influenced by latent1 and self (y5[t-1])
    y5[0] = np.random.normal()
    for t in range(1, n):
        y5[t] = 0.6 * latent1[t] + 0.5 * y5[t-1] + np.random.normal()

    # Introduce NaN values randomly
    nan_indices = np.random.choice(n, size=int(n * nan_ratio), replace=False)
    for y in [y1, y2, y3, y4, y5]:
        y[nan_indices] = np.nan

    # Combine into DataFrame
    data = pd.DataFrame({'y1': y1, 'y2': y2, 'y3': y3, 'y4': y4, 'y5': y5})

    # Ground truth causal matrix
        # Ground truth causal matrix
    ground_truth = np.array([
        [['', ''], ['<--', ''], ['', '-->'], ['', ''], ['', '']],
        [['-->', ''], ['', ''], ['', ''], ['', '-->'], ['o-o', '']],
        [['', ''], ['', ''], ['', ''], ['-->', ''], ['', '']],
        [['', ''], ['', ''], ['<--', ''], ['', ''], ['', '']],
        [['', ''], ['o-o', ''], ['', ''], ['', ''], ['', '-->']]
    ])

    return data, ground_truth

def generate_dataset(n, nan_ratio, phi=0.8):
    
    # Initialize latent confounder
    latent = np.zeros(n)
    latent[0] = np.random.normal()
    for t in range(1, n):
        latent[t] = phi * latent[t-1] + np.random.normal()

    # Initialize variables y1 to y5
    y1 = np.zeros(n)
    y2 = np.zeros(n)
    y3 = np.zeros(n)
    y4 = np.zeros(n)
    y5 = np.zeros(n)

    # Generate y1 using AR(1) with non-linear transformation
    y1[0] = np.random.normal()
    for t in range(1, n):
        y1[t] = 0.9 * y1[t-1] + np.random.normal()
    y1 = np.log(np.abs(y1) + 1) + np.random.normal(scale=0.1, size=n)

    # Generate y2 influenced by latent and y1
    y2[0] = np.random.normal()
    for t in range(1, n):
        y2[t] = 0.8 * latent[t] + 0.3 * np.tanh(y1[t]) + np.random.normal()

    # Generate y3 influenced by latent and y2
    y3[0] = np.random.normal()
    for t in range(1, n):
        y3[t] = 0.6 * latent[t] + np.random.normal()

    # Generate y4 influenced by lagged y1 and y3
    y4[0] = np.random.normal()
    for t in range(1, n):
        y4[t] = 0.4 * np.roll(y1, 1)[t] + 0.9 * np.sqrt(np.abs(y3[t])) + np.random.normal()

    # Generate y5 influenced by latent, y2, and lagged y4
    y5[0] = np.random.normal()
    for t in range(1, n):
        y5[t] = 0.7 * latent[t] + 0.5 * np.log(np.abs(y2[t]) + 1) + 0.6 * np.roll(y4, 1)[t] + np.random.normal()

    # Introduce NaN values randomly
    nan_indices = np.random.choice(n, size=int(n * nan_ratio), replace=False)
    for y in [y1, y2, y3, y4, y5]:
        y[nan_indices] = np.nan

    # Combine into DataFrame
    data = pd.DataFrame({'y0': y1, 'y1': y2, 'y2': y3, 'y3': y4, 'y4': y5})

    # Ground truth causal matrix
    ground_truth = np.array([
        [['', ''], ['<--', ''], ['', '-->'], ['', ''], ['', '']],
        [['-->', ''], ['', ''], ['', ''], ['', '-->'], ['o-o', '']],
        [['', ''], ['', ''], ['', ''], ['-->', ''], ['', '']],
        [['', ''], ['', ''], ['<--', ''], ['', ''], ['', '']],
        [['', ''], ['o-o', ''], ['', ''], ['', ''], ['', '-->']]
    ])

    return data, ground_truth

In [ ]:
# Function to generate the dataset and its ground truth causal matrix
def generate_causal_dataset(n, nan_ratio=0.1, phi_latent1=0.8, phi_latent2=0.7):
    # Initialize latent confounders
    latent1 = np.zeros(n)
    latent2 = np.zeros(n)
    latent1[0] = np.random.laplace()  # Non-Gaussian distribution (Laplace)
    latent2[0] = np.random.laplace()  # Non-Gaussian distribution (Laplace)

    # Generate latent confounders using AR(1) processes with Laplace distribution
    for t in range(1, n):
        latent1[t] = phi_latent1 * latent1[t-1] + np.random.laplace()
        latent2[t] = phi_latent2 * latent2[t-1] + np.random.laplace()

    # Initialize variables y1 to y5
    y1 = np.zeros(n)
    y2 = np.zeros(n)
    y3 = np.zeros(n)
    y4 = np.zeros(n)
    y5 = np.zeros(n)

    # Generate y1 influenced by latent1 and itself (Non-linear and Non-Gaussian)
    y1[0] = np.random.laplace()
    for t in range(1, n):
        y1[t] = np.tanh(0.6 * y1[t-1]) + 0.4 * np.sin(latent1[t]) + np.random.laplace()

    # Generate y2 influenced by latent1 and itself (Non-linear and Non-Gaussian)
    y2[0] = np.random.laplace()
    for t in range(1, n):
        y2[t] = np.tanh(0.7 * y2[t-1]) + 0.5 * np.exp(latent1[t]) + np.random.laplace()

    # Generate y3 influenced by y2[t] and itself (Non-linear and Non-Gaussian)
    y3[0] = np.random.laplace()
    for t in range(1, n):
        y3[t] = np.tanh(0.8 * y3[t-1]) + 0.6 * np.sin(y2[t]) + 0.5 * np.tanh(latent2[t]) + np.random.laplace()

    # Generate y4 influenced by y3[t-1] (Non-linear and Non-Gaussian)
    y4[0] = np.random.laplace()
    for t in range(1, n):
        y4[t] = 0.5 * np.tanh(y3[t-1]) + np.random.laplace()

    # Generate y5 influenced by latent2 and itself (Non-linear and Non-Gaussian)
    y5[0] = np.random.laplace()
    for t in range(1, n):
        y5[t] = np.tanh(0.7 * y5[t-1]) + 0.5 * np.log(np.abs(latent2[t]) + 1) + np.random.laplace()

    # Introduce NaN values randomly
    nan_indices = np.random.choice(n, size=int(n * nan_ratio), replace=False)
    for y in [y1, y2, y3, y4, y5]:
        y[nan_indices] = np.nan

    # Combine into DataFrame
    data = pd.DataFrame({'y1': y1, 'y2': y2, 'y3': y3, 'y4': y4, 'y5': y5})

    # Ground truth causal matrix
    ground_truth = np.array([
        [['', '-->'], ['o-o', ''], ['', ''], ['', ''], ['', '']],
        [['o-o', ''], ['', '-->'], ['-->', ''], ['', ''], ['', '']],
        [['', ''], ['<--', ''], ['', '-->'], ['', '-->'], ['o-o', '']],
        [['', ''], ['', ''], ['', '<--'], ['', ''], ['', '']],
        [['', ''], ['', ''], ['o-o', ''], ['', ''], ['', '-->']]
    ])

    return data, ground_truth


10 nodes

In [4]:
import numpy as np
import pandas as pd

# Function to generate the dataset and its ground truth causal matrix for a 10-node system
def generate_causal_dataset(n, nan_ratio=0.1, phi_latent=0.8):
    # Initialize latent confounder
    latent1 = np.zeros(n)
    latent1[0] = np.random.normal()

    # Generate latent confounder using an AR(1) process
    for t in range(1, n):
        latent1[t] = phi_latent * latent1[t - 1] + np.random.normal()

    # Initialize variables y1 to y10
    y = {f'y{i}': np.zeros(n) for i in range(1, 11)}

    # Define the causal relationships
    # y2 influenced by latent1
    y['y2'][0] = np.random.normal()
    for t in range(1, n):
        y['y2'][t] = 0.6 * latent1[t] + np.random.normal()

    # y1 influenced by y2 (contemporaneous)
    y['y1'][0] = np.random.normal()
    for t in range(1, n):
        y['y1'][t] = 0.5 * y['y2'][t] + np.random.normal()

    # y3 influenced by y1[t-1]
    y['y3'][0] = np.random.normal()
    for t in range(1, n):
        y['y3'][t] = 0.7 * y['y1'][t - 1] + np.random.normal()

    # y4 influenced by y2[t-1] and y3[t] (contemporaneous)
    y['y4'][0] = np.random.normal()
    for t in range(1, n):
        y['y4'][t] = 0.5 * y['y2'][t - 1] + 0.8 * y['y3'][t] + np.random.normal()

    # y5 influenced by latent1 and self (y5[t-1])
    y['y5'][0] = np.random.normal()
    for t in range(1, n):
        y['y5'][t] = 0.6 * latent1[t] + 0.5 * y['y5'][t - 1] + np.random.normal()

    # Additional nodes y6 to y10 with new relationships
    # y6 influenced by y5[t-1] and latent1
    y['y6'][0] = np.random.normal()
    for t in range(1, n):
        y['y6'][t] = 0.7 * y['y5'][t - 1] + 0.4 * latent1[t] + np.random.normal()

    # y7 influenced by y6[t] and y2[t-1]
    y['y7'][0] = np.random.normal()
    for t in range(1, n):
        y['y7'][t] = 0.5 * y['y6'][t] + 0.3 * y['y2'][t - 1] + np.random.normal()

    # y8 influenced by y3[t-1] and y7[t]
    y['y8'][0] = np.random.normal()
    for t in range(1, n):
        y['y8'][t] = 0.6 * y['y3'][t - 1] + 0.5 * y['y7'][t] + np.random.normal()

    # y9 influenced by y4[t] and latent1
    y['y9'][0] = np.random.normal()
    for t in range(1, n):
        y['y9'][t] = 0.4 * y['y4'][t] + 0.6 * latent1[t] + np.random.normal()

    # y10 influenced by y8[t] and y9[t-1]
    y['y10'][0] = np.random.normal()
    for t in range(1, n):
        y['y10'][t] = 0.5 * y['y8'][t] + 0.7 * y['y9'][t - 1] + np.random.normal()

    # Introduce NaN values randomly
    nan_indices = np.random.choice(n, size=int(n * nan_ratio), replace=False)
    for i in range(1, 11):
        y[f'y{i}'][nan_indices] = np.nan

    # Combine into DataFrame
    data = pd.DataFrame(y)

    # Define ground truth causal matrix (updated for 10 nodes)
    ground_truth = np.array([
        [['', ''], ['<--', ''], ['', '-->'], ['', ''], ['', ''], ['', ''], ['', ''], ['', ''], ['', ''], ['', '']],
        [['-->', ''], ['', ''], ['', ''], ['', '-->'], ['o-o', ''], ['o-o', ''], ['', '-->'], ['', ''], ['o-o', ''], ['', '']],
        [['', '<--'], ['', ''], ['', ''], ['-->', ''], ['', ''], ['', ''], ['', ''], ['', '-->'], ['', ''], ['', '']],
        [['', ''], ['', '<--'], ['<--', ''], ['', ''], ['', ''], ['', ''], ['', ''], ['', ''], ['-->', ''], ['', '']],
        [['', ''], ['o-o', ''], ['', ''], ['', ''], ['', '-->'], ['', '-->'], ['', ''], ['', ''], ['o-o', ''], ['', '']],
        [['', ''], ['o-o', ''], ['', ''], ['', ''], ['', '<--'], ['', ''], ['-->', ''], ['', ''], ['o-o', ''], ['', '']],
        [['', ''], ['', '<--'], ['', ''], ['', ''], ['', ''], ['<--', ''], ['', ''], ['-->', ''], ['', ''], ['', '']],
        [['', ''], ['', ''], ['', '<--'], ['', ''], ['', ''], ['', ''], ['<--', ''], ['', ''], ['', ''], ['-->', '']],
        [['', ''], ['o-o', ''], ['', ''], ['<--', ''], ['o-o', ''], ['o-o', ''], ['', ''], ['', ''], ['', ''], ['', '-->']],
        [['', ''], ['', ''], ['', ''], ['', ''], ['', ''], ['', ''], ['', ''], ['<--', ''], ['', '<--'], ['', '']]
    ])

    return data, ground_truth

generate_causal_dataset(50)


         y1        y2        y3        y4        y5        y6        y7  \
0  0.262625  0.439522 -0.514745  0.681817 -0.941773  1.321716  0.761944   
1 -0.630616 -1.473605 -0.507248 -1.417785 -0.720644 -2.729940 -0.747854   
2 -2.128621 -0.627970  0.329550 -1.375481 -0.660218  0.706367 -0.454569   
3 -0.748819 -0.968439 -0.809640 -0.685816  1.057344 -0.308763 -1.010588   
4  1.861781  1.172340  0.175683 -0.547207  1.418841  1.944589 -1.533666   

         y8        y9       y10  
0  0.894747  0.542049  0.637034  
1  0.921450 -1.897056  1.450709  
2 -1.217333 -0.721907 -1.789402  
3  0.564964 -0.197513 -0.758310  
4 -0.146895 -1.126502 -0.678719  


(          y1        y2        y3        y4        y5        y6        y7  \
 0   0.262625  0.439522 -0.514745  0.681817 -0.941773  1.321716  0.761944   
 1  -0.630616 -1.473605 -0.507248 -1.417785 -0.720644 -2.729940 -0.747854   
 2  -2.128621 -0.627970  0.329550 -1.375481 -0.660218  0.706367 -0.454569   
 3  -0.748819 -0.968439 -0.809640 -0.685816  1.057344 -0.308763 -1.010588   
 4   1.861781  1.172340  0.175683 -0.547207  1.418841  1.944589 -1.533666   
 5   0.224707 -1.207436  0.535360  2.279898  0.096602  0.034396 -0.396807   
 6  -0.689758 -1.292714  1.040664  1.524622 -2.251377 -0.823914 -2.009425   
 7   0.249865 -0.361779 -0.339004  0.314539  1.130195 -0.617462  0.064076   
 8   2.500232  2.618285  1.514138  0.449225  2.415574  0.248001 -1.964231   
 9   1.441422  1.452818  2.251915  2.795177  1.616762  2.974181  3.800008   
 10  0.049156 -0.022579  1.705225  0.429595  2.388770 -1.366967  0.155301   
 11 -1.054129 -0.577903 -1.199180 -1.205905  0.765448  2.363444  1.525745   

15 nodes

In [5]:
import numpy as np
import pandas as pd

# Function to generate the dataset and its ground truth causal matrix for a 15-node system
def generate_causal_dataset(n, nan_ratio=0.1, phi_latent=0.8):
    # Initialize latent confounder
    latent1 = np.zeros(n)
    latent1[0] = np.random.normal()

    # Generate latent confounder using an AR(1) process
    for t in range(1, n):
        latent1[t] = phi_latent * latent1[t - 1] + np.random.normal()

    # Initialize variables y1 to y15
    y = {f'y{i}': np.zeros(n) for i in range(1, 16)}

    # Define the causal relationships
    # y2 influenced by latent1
    y['y2'][0] = np.random.normal()
    for t in range(1, n):
        y['y2'][t] = 0.6 * latent1[t] + np.random.normal()

    # y1 influenced by y2 (contemporaneous)
    y['y1'][0] = np.random.normal()
    for t in range(1, n):
        y['y1'][t] = 0.5 * y['y2'][t] + np.random.normal()

    # y3 influenced by y1[t-1]
    y['y3'][0] = np.random.normal()
    for t in range(1, n):
        y['y3'][t] = 0.7 * y['y1'][t - 1] + np.random.normal()

    # y4 influenced by y2[t-1] and y3[t] (contemporaneous)
    y['y4'][0] = np.random.normal()
    for t in range(1, n):
        y['y4'][t] = 0.5 * y['y2'][t - 1] + 0.8 * y['y3'][t] + np.random.normal()

    # y5 influenced by latent1 and self (y5[t-1])
    y['y5'][0] = np.random.normal()
    for t in range(1, n):
        y['y5'][t] = 0.6 * latent1[t] + 0.5 * y['y5'][t - 1] + np.random.normal()

    # Additional nodes y6 to y15 with new relationships
    # y6 influenced by y5[t-1] and latent1
    y['y6'][0] = np.random.normal()
    for t in range(1, n):
        y['y6'][t] = 0.7 * y['y5'][t - 1] + 0.4 * latent1[t] + np.random.normal()

    # y7 influenced by y6[t] and y2[t-1]
    y['y7'][0] = np.random.normal()
    for t in range(1, n):
        y['y7'][t] = 0.5 * y['y6'][t] + 0.3 * y['y2'][t - 1] + np.random.normal()

    # y8 influenced by y3[t-1] and y7[t]
    y['y8'][0] = np.random.normal()
    for t in range(1, n):
        y['y8'][t] = 0.6 * y['y3'][t - 1] + 0.5 * y['y7'][t] + np.random.normal()

    # y9 influenced by y4[t] and latent1
    y['y9'][0] = np.random.normal()
    for t in range(1, n):
        y['y9'][t] = 0.4 * y['y4'][t] + 0.6 * latent1[t] + np.random.normal()

    # y10 influenced by y8[t] and y9[t-1]
    y['y10'][0] = np.random.normal()
    for t in range(1, n):
        y['y10'][t] = 0.5 * y['y8'][t] + 0.7 * y['y9'][t - 1] + np.random.normal()




    # y11 influenced by y9 and latent1
    y['y11'][0] = np.random.normal()
    for t in range(1, n):
        y['y11'][t] = 0.3 * y['y9'][t] + 0.5 * latent1[t] + np.random.normal()

    # y12 influenced by y10[t-1] and y5[t]
    y['y12'][0] = np.random.normal()
    for t in range(1, n):
        y['y12'][t] = 0.4 * y['y10'][t - 1] + 0.6 * y['y5'][t] + np.random.normal()

    # y13 influenced by y12 and y7[t-1]
    y['y13'][0] = np.random.normal()
    for t in range(1, n):
        y['y13'][t] = 0.5 * y['y12'][t] + 0.3 * y['y7'][t - 1] + np.random.normal()

    # y14 influenced by y8[t] and y13[t-1]
    y['y14'][0] = np.random.normal()
    for t in range(1, n):
        y['y14'][t] = 0.6 * y['y8'][t] + 0.4 * y['y13'][t - 1] + np.random.normal()

    # y15 influenced by latent1 and y14[t]
    y['y15'][0] = np.random.normal()
    for t in range(1, n):
        y['y15'][t] = 0.7 * latent1[t] + 0.3 * y['y14'][t] + np.random.normal()

    # Introduce NaN values randomly
    nan_indices = np.random.choice(n, size=int(n * nan_ratio), replace=False)
    for i in range(1, 16):
        y[f'y{i}'][nan_indices] = np.nan

    # Combine into DataFrame
    data = pd.DataFrame(y)

    # Define ground truth causal matrix (updated for 15 nodes)
    ground_truth = np.array([
        [['', ''], ['<--', ''], ['', '-->'], ['', ''], ['', ''], ['', ''], ['', ''], ['', ''], ['', ''], ['', ''], ['', ''], ['', ''], ['', ''], ['', ''], ['', '']],
        [['-->', ''], ['', ''], ['', ''], ['', '-->'], ['o-o', ''], ['o-o', ''], ['', '-->'], ['', ''], ['o-o', ''], ['', ''], ['o-o', ''], ['', ''], ['', ''], ['', ''], ['o-o', '']],
        [['', '<--'], ['', ''], ['', ''], ['-->', ''], ['', ''], ['', ''], ['', ''], ['', '-->'], ['', ''], ['', ''], ['', ''], ['', ''], ['', ''], ['', ''], ['', '']],
        [['', ''], ['', '<--'], ['<--', ''], ['', ''], ['', ''], ['', ''], ['', ''], ['', ''], ['-->', ''], ['', ''], ['', ''], ['', ''], ['', ''], ['', ''], ['', '']],
        [['', ''], ['o-o', ''], ['', ''], ['', ''], ['', '-->'], ['', '-->'], ['', ''], ['', ''], ['o-o', ''], ['', ''], ['o-o', ''], ['-->', ''], ['', ''], ['', ''], ['o-o', '']],
        [['', ''], ['o-o', ''], ['', ''], ['', ''], ['', '<--'], ['', ''], ['-->', ''], ['', ''], ['o-o', ''], ['', ''], ['o-o', ''], ['', ''], ['', ''], ['', ''], ['o-o', '']],
        [['', ''], ['', '<--'], ['', ''], ['', ''], ['', ''], ['<--', ''], ['', ''], ['-->', ''], ['', ''], ['', ''], ['', ''], ['', '-->'], ['', ''], ['', ''], ['', '']],
        [['', ''], ['', ''], ['', '<--'], ['', ''], ['', ''], ['', ''], ['<--', ''], ['', ''], ['', ''], ['-->', ''], ['', ''], ['', ''], ['', ''], ['-->', ''], ['', '']],
        [['', ''], ['o-o', ''], ['', ''], ['<--', ''], ['o-o', ''], ['o-o', ''], ['', ''], ['', ''], ['', ''], ['', '-->'], ['-->', ''], ['', ''], ['', ''], ['', ''], ['o-o', '']],
        [['', ''], ['', ''], ['', ''], ['', ''], ['', ''], ['', ''], ['', ''], ['<--', ''], ['', '<--'], ['', ''], ['', ''], ['', '-->'], ['', ''], ['', ''], ['', '']],
        [['', ''], ['o-o', ''], ['', ''], ['', ''], ['o-o', ''], ['o-o', ''], ['', ''], ['', ''], ['<--', ''], ['', ''], ['', ''], ['', ''], ['', ''], ['', ''], ['o-o', '']],
        [['', ''], ['', ''], ['', ''], ['', ''], ['<--', ''], ['', ''], ['', ''], ['', ''], ['', ''], ['', '<--'], ['', ''], ['', ''], ['-->', ''], ['', ''], ['', '']],
        [['', ''], ['', ''], ['', ''], ['', ''], ['', ''], ['', ''], ['', '<--'], ['', ''], ['', ''], ['', ''], ['', ''], ['<--', ''], ['', ''], ['', '-->'], ['', '']],
        [['', ''], ['', ''], ['', ''], ['', ''], ['', ''], ['', ''], ['', ''], ['<--', ''], ['', ''], ['', ''], ['', ''], ['', ''], ['', '<--'], ['', ''], ['-->', '']],
        [['', ''], ['o-o', ''], ['', ''], ['', ''], ['o-o', ''], ['o-o', ''], ['', ''], ['', ''], ['o-o', ''], ['', ''], ['o-o', ''], ['', ''], ['', ''], ['<--', ''], ['', '']]
    ])

    return data, ground_truth
